# Eurostat
Code to collect and process eurostat data to create the following indicators:

* Private sector R&D workforce
* Business Enterprise R&D (BERD)
* Share if hugh growth firms 

Raw data collected using the Eurostat API via the `EuroStat API Client` python package (https://pypi.org/project/eurostatapiclient/).

## Preamble

In [ ]:
%load_ext autoreload
%autoreload 2

from eurostatapiclient import EurostatAPIClient

import numpy as np
import pandas as pd

from beis_indicators.geo.nuts import auto_nuts2_uk
import beis_indicators.geo.nuts as geo_nuts

In [ ]:
VERSION = 'v2.1'
FORMAT = 'json'
LANGUAGE = 'en'

In [ ]:
client = EurostatAPIClient(VERSION, FORMAT, LANGUAGE)

### Mappings

In [ ]:
#using the eurostat code labels
vars_map = {
    'EUR_HAB': 'Euro per inhabitant',
    'MIO_EUR': 'Million euro',
    'FTE': 'Full-time equivalent (FTE)',
    'HC': 'Head count', 
    'PC_ACT_FTE': 'Percentage of active population - numerator in full-time equivalent (FTE)',
    'PC_ACT_HC': 'Percentage of active population - numerator in head count (HC)',
    'PPS_HAB': 'Purchasing power standard (PPS) per inhabitant'
}

## Data Collection, Processing & Transformation

This section is made up with three sections- one for each indicator. Each section is broken down in the following steps:

* Use the python package to pull down flattened data by entering using a query & put into a dataframe
* Collect the subset for UK NUTS2 regions
* Replace the codes with the label associated 
* Data is transformed into a pivot table to output the desired format

### Private sector R&D workforce data

#### Head Count

In [ ]:
#pull in data
data_priv_nuts2 = client.get_dataset('rd_p_persreg?sinceTimePeriod=2007&geoLevel=nuts2&precision=1&sex=T&sectperf=BES&prof_pos=TOTAL&unit=HC')

print(data_priv_nuts2.label)

dataframe_priv_nuts2 = data_priv_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
dataframe_priv_nuts2_uk = dataframe_priv_nuts2[dataframe_priv_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
dataframe_priv_nuts2_uk['time'] = dataframe_priv_nuts2_uk['time'].astype(int)
dataframe_priv_nuts2_uk['unit'] = dataframe_priv_nuts2_uk['unit'].map(vars_map)

In [ ]:
#pivot table
d_priv = dataframe_priv_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo')

In [ ]:
d_priv['Head count'] = pd.to_numeric(d_priv['Head count'], downcast='integer')

In [ ]:
d_priv.columns

In [ ]:
d_priv.reset_index(inplace=True)
d_priv.columns = ['nuts_id', 'year', 'eurostat_private_rd_headcount_workforce_data']
d_priv = auto_nuts2_uk(d_priv)

In [ ]:
d_priv = d_priv[['year','nuts_id', 'nuts_year_spec', 'eurostat_private_rd_headcount_workforce_data']]

In [ ]:
#save data
d_priv.to_csv('../../data/processed/eurostat/eurostat_private_rd_headcount_workforce_data.nuts2.csv',index=False)

#### Full Time Equivalent (FTE)

In [ ]:
#pull in data
data_priv_nuts2 = client.get_dataset('rd_p_persreg?sinceTimePeriod=2007&geoLevel=nuts2&precision=6&sex=T&sectperf=BES&prof_pos=TOTAL&unit=FTE')

print(data_priv_nuts2.label)

dataframe_priv_nuts2 = data_priv_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
dataframe_priv_nuts2_uk = dataframe_priv_nuts2[dataframe_priv_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
dataframe_priv_nuts2_uk['time'] = dataframe_priv_nuts2_uk['time'].astype(int)
dataframe_priv_nuts2_uk['unit'] = dataframe_priv_nuts2_uk['unit'].map(vars_map)

In [ ]:
#pivot table
d_priv = dataframe_priv_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo')

In [ ]:
d_priv.columns

In [ ]:
d_priv.reset_index(inplace=True)
d_priv.columns = ['nuts_id', 'year', 'eurostat_private_rd_fte_workforce_data']
d_priv = auto_nuts2_uk(d_priv)

In [ ]:
d_priv = d_priv[['year','nuts_id', 'nuts_year_spec', 'eurostat_private_rd_fte_workforce_data']]

In [ ]:
#save data
d_priv.to_csv('../../data/processed/eurostat/eurostat_private_rd_fte_workforce_data.nuts2.csv', index=False)

### Business Enterprise R&D (BERD) data

In [ ]:
#pull in data
data_berd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2007&geoLevel=nuts2&precision=6&sectperf=BES&unit=MIO_EUR')

print(data_berd_nuts2.label)

dataframe_berd_nuts2 = data_berd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
dataframe_berd_nuts2_uk = dataframe_berd_nuts2[dataframe_berd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
dataframe_berd_nuts2_uk['time'] = dataframe_berd_nuts2_uk['time'].astype(int)
dataframe_berd_nuts2_uk['unit'] = dataframe_berd_nuts2_uk['unit'].map(vars_map)

In [ ]:
#pivot table
d_berd = dataframe_berd_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo')

In [ ]:
d_berd['euros'] = d_berd['Million euro'] * 1000000.00
d_berd.drop(columns=['Million euro'], inplace=True)

In [ ]:
d_berd.columns

In [ ]:
d_berd.reset_index(inplace=True)
d_berd.columns = ['nuts_id', 'year', 'eurostat_berd_data']
d_berd = auto_nuts2_uk(d_berd)

In [ ]:
d_berd = d_berd[['year','nuts_id', 'nuts_year_spec', 'eurostat_berd_data']]

In [ ]:
#save data
d_berd.to_csv('../../data/processed/eurostat/eurostat_berd_data.nuts2.csv',index=False)

### Private non-profit (PNPERD) data

In [ ]:
#pull in data
data_pnpberd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2007&geoLevel=nuts2&precision=6&sectperf=PNP&unit=MIO_EUR')

print(data_pnpberd_nuts2.label)

df_pnpberd_nuts2 = data_pnpberd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
df_pnpberd_nuts2_uk = df_pnpberd_nuts2[df_pnpberd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
df_pnpberd_nuts2_uk['time'] = df_pnpberd_nuts2_uk['time'].astype(int)
df_pnpberd_nuts2_uk['unit'] = df_pnpberd_nuts2_uk['unit'].map(vars_map)

In [ ]:
d_pnpberd = df_pnpberd_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo')

In [ ]:
d_pnpberd['euros'] = d_pnpberd['Million euro'] * 1000000.00
d_pnpberd.drop(columns=['Million euro'], inplace=True)

In [ ]:
d_pnpberd.columns

In [ ]:
d_pnpberd.reset_index(inplace=True)
d_pnpberd.columns = ['nuts_id', 'year', 'eurostat_private_non_profit_rd_workforce_data']
d_pnpberd = auto_nuts2_uk(d_pnpberd)

In [ ]:
d_pnpberd = d_pnpberd[['year','nuts_id', 'nuts_year_spec', 'eurostat_private_non_profit_rd_workforce_data']]

In [ ]:
d_pnpberd.to_csv('../../data/processed/eurostat/eurostat_private_non_profit_rd_workforce_data.nuts2.csv', index=False)

### Higher Education Performed R&D expenditure (HERD)

In [ ]:
#pull in data
data_herd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2007&geoLevel=nuts2&precision=6&sectperf=HES&unit=MIO_EUR')

print(data_herd_nuts2.label)

df_herd_nuts2 = data_herd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
df_herd_nuts2_uk = df_herd_nuts2[df_herd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
df_herd_nuts2_uk['time'] = df_herd_nuts2_uk['time'].astype(int)
df_herd_nuts2_uk['unit'] = df_herd_nuts2_uk['unit'].map(vars_map)

In [ ]:
d_herd = df_herd_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo')

In [ ]:
d_herd['euros'] = d_herd['Million euro'] * 1000000.00
d_herd.drop(columns=['Million euro'], inplace=True)

In [ ]:
d_herd.columns

In [ ]:
d_herd.reset_index(inplace=True)
d_herd.columns = ['nuts_id', 'year', 'eurostat_higher_ed_rd_workforce_data']
d_herd = auto_nuts2_uk(d_herd)

In [ ]:
d_herd = d_herd[['year','nuts_id', 'nuts_year_spec', 'eurostat_higher_ed_rd_workforce_data']]

In [ ]:
d_herd.to_csv('../../data/processed/eurostat/eurostat_higher_ed_rd_workforce_data.nuts2.csv',index=False)

### Government Performed R&D Expenditure (GovERD)

In [ ]:
#pull in data
data_goverd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2007&geoLevel=nuts2&precision=2&sectperf=GOV&unit=MIO_EUR')

print(data_goverd_nuts2.label)

df_goverd_nuts2 = data_goverd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
df_goverd_nuts2_uk = df_goverd_nuts2[df_goverd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
df_goverd_nuts2_uk['time'] = df_goverd_nuts2_uk['time'].astype(int)
df_goverd_nuts2_uk['unit'] = df_goverd_nuts2_uk['unit'].map(vars_map)

In [ ]:
d_goverd = df_goverd_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo')

In [ ]:
d_goverd['euros'] = d_goverd['Million euro'] * 1000000.00
d_goverd.drop(columns=['Million euro'], inplace=True)

In [ ]:
d_goverd.columns

In [ ]:
d_goverd.reset_index(inplace=True)
d_goverd.columns = ['nuts_id', 'year', 'eurostat_gov_rd_workforce_data']
d_goverd = auto_nuts2_uk(d_goverd)

In [ ]:
d_goverd = d_goverd[['year','nuts_id', 'nuts_year_spec', 'eurostat_gov_rd_workforce_data']]

In [ ]:
d_goverd.to_csv('../../data/processed/eurostat/eurostat_gov_rd_workforce_data.nuts2.csv', index=False)

### Private Household Income

In [ ]:
data_house_nuts2 = client.get_dataset('tgs00036?sinceTimePeriod=2007&precision=2&unit=PPS_HAB')

print(data_house_nuts2.label)

data_house_nuts2 = data_house_nuts2.to_dataframe()

In [ ]:
# data_house_nuts2

In [ ]:
#UK NUTS2 regions subset
data_house_nuts2_uk = data_house_nuts2[data_house_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
data_house_nuts2_uk['time'] = data_house_nuts2_uk['time'].astype(int)
data_house_nuts2_uk['unit'] = data_house_nuts2_uk['unit'].map(vars_map)

In [ ]:
# data_house_nuts2_uk

In [ ]:
d_house = data_house_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo')

In [ ]:
d_house['Purchasing power standard (PPS) per inhabitant'] = d_house['Purchasing power standard (PPS) per inhabitant'].round(2)

In [ ]:
d_house.columns

In [ ]:
d_house.reset_index(inplace=True)
d_house.columns = ['nuts_id', 'year', 'eurostat_private_households_income']
d_house = auto_nuts2_uk(d_house)

In [ ]:
d_house = d_house[['year','nuts_id', 'nuts_year_spec', 'eurostat_private_households_income']]

In [ ]:
d_house.to_csv('../../data/processed/eurostat/eurostat_private_households_income.nuts2.csv', index=False)

### Share of high growth firms

In [ ]:
#pull in data
data_share_nuts2 = client.get_dataset('bd_hgnace2_r3?sinceTimePeriod=2007&geoLevel=nuts2&precision=1&indic_sb=V97460&nace_r2=B-E&nace_r2=B-S_X_K642&nace_r2=F&nace_r2=G&nace_r2=H&nace_r2=I&nace_r2=J&nace_r2=K_L_X_K642&nace_r2=M_N&nace_r2=P_Q&nace_r2=R_S')

print(data_share_nuts2.label)

dataframe_share_nuts2 = data_share_nuts2.to_dataframe()

In [ ]:
dataframe_share_nuts2[dataframe_share_nuts2['geo'] == 'UKK1']

In [ ]:
#UK NUTS2 regions subset

dataframe_share_nuts2_uk = dataframe_share_nuts2[dataframe_share_nuts2['geo'].str.contains('UK')]

In [ ]:
dataframe_share_nuts2_uk

In [ ]:
dataframe_share_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'indic_sb',
               values = 'values')

Note: Does not seem to be UK NUTS2 values for this dataset